In [1]:
import lightgbm as lgbm
import numpy as np
import pandas as pd

# define data types for the sales data
numCols = [f"d_{day}" for day in range(1, 1914)]

# Define all categorical columns
catCols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']

# Define the correct data types for "sales_train_validation.csv"
dtype = {numCol: "float32" for numCol in numCols}
dtype.update({catCol: "category" for catCol in catCols if catCol != "id"})

# read the sales data
sales_data = pd.read_csv("/media/hhew0002/f0df6edb-45fe-4416-8076-34757a0abceb/hhew0002/Academic/Competitions/M5 Competition/data/sales_train_validation.csv", usecols=catCols + numCols, dtype=dtype)
sales_data = sales_data.drop(columns=["item_id", "dept_id", "store_id"])

In [2]:
# read the calendar data
calendarDTypes = {"event_name_1": "category",
                  "event_name_2": "category",
                  "event_type_1": "category",
                  "event_type_2": "category",
                  "weekday": "category",
                  'wm_yr_wk': 'int16',
                  "wday": "category",
                  "month": "category",
                  "year": "int16",
                  "snap_CA": "category",
                  'snap_TX': 'category',
                  'snap_WI': 'category' }
calendar_data = pd.read_csv("/media/hhew0002/f0df6edb-45fe-4416-8076-34757a0abceb/hhew0002/Academic/Competitions/M5 Competition/data/calendar.csv", dtype=calendarDTypes)

# remove the last 28 points from the calendar data
horizon = 28
calendar_data = calendar_data.iloc[:-horizon, :]


for col, colDType in calendarDTypes.items():
    if colDType == "category":
        calendar_data[col] = calendar_data[col].cat.codes.astype("int16")
        calendar_data[col] -= calendar_data[col].min()

def add_weekend(row):
    if row["wday"] == 1 or row["wday"] == 2:
        return 1
    else:
        return 0

calendar_data['weekend'] = calendar_data.apply (lambda row: add_weekend(row), axis=1)
calendar_data = calendar_data.drop(columns=["date", "wm_yr_wk", "year", "weekday", "wday"])


In [8]:
# read the price data
price_data = pd.read_csv("/media/hhew0002/f0df6edb-45fe-4416-8076-34757a0abceb/hhew0002/Academic/Competitions/M5 Competition/data/Item-prices.txt", header=None)
price_data = price_data.iloc[:,:-horizon]

column_names = [f"d_{day}" for day in range(1, 1942)]
price_data.columns = column_names

# add the item id column
price_data["id"] = sales_data["id"]
# melt the price data
price_data_melted = pd.melt(price_data, id_vars=['id'], var_name="d", value_name="price")


In [4]:
# embed the sales data to have a lag length of 10
data = sales_data.iloc[0:10, :]
data = pd.melt(data, id_vars=['id', 'cat_id', 'state_id'], var_name="d", value_name="sales")

for lags in range(1,11):
    data['lag' + str(lags)] = data[["id","sales"]].groupby("id")["sales"].shift(lags)

In [6]:
# merge the calendar data into the sales data
full_training_data = data.merge(calendar_data, on="d", copy=False)
full_training_data = full_training_data.dropna()

# merge the price data into the sales data
full_training_data = full_training_data.merge(price_data_melted, on=["id","d"], copy=False)

# create only one column for the snap features
def add_snap(row):
    if row["state_id"] == "CA":
        return row["snap_CA"]
    elif row["state_id"] == "TX":
        return row["snap_TX"]
    else:
        return row["snap_WI"]

full_training_data['snap'] = full_training_data.apply (lambda row: add_snap(row), axis=1)
full_training_data = full_training_data.drop(columns=["id", "d", "snap_CA", "snap_WI", "snap_TX"])
full_training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19030 entries, 0 to 19029
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   cat_id        19030 non-null  category
 1   state_id      19030 non-null  category
 2   sales         19030 non-null  float32 
 3   lag1          19030 non-null  float32 
 4   lag2          19030 non-null  float32 
 5   lag3          19030 non-null  float32 
 6   lag4          19030 non-null  float32 
 7   lag5          19030 non-null  float32 
 8   lag6          19030 non-null  float32 
 9   lag7          19030 non-null  float32 
 10  lag8          19030 non-null  float32 
 11  lag9          19030 non-null  float32 
 12  lag10         19030 non-null  float32 
 13  month         19030 non-null  int16   
 14  event_name_1  19030 non-null  int16   
 15  event_type_1  19030 non-null  int16   
 16  event_name_2  19030 non-null  int16   
 17  event_type_2  19030 non-null  int16   
 18  weeken

In [22]:
# create the training x and y data
full_training_data_x = full_training_data.drop(columns=["sales"])
full_training_data_y = full_training_data.sales

# seperate the training and validation data
np.random.seed(777)

# Define categorical features
cat_features = ['cat_id', 'state_id', 'weekend', 'month', 'event_name_1', 'event_name_2', 'event_type_1', 'event_type_2', 'snap']

# validation data
validation_indices = np.random.choice(full_training_data_x.index.values, 2_000_000, replace = False)
training_indices = np.setdiff1d(full_training_data_x.index.values, validation_indices)

training_data = lgbm.Dataset(full_training_data_x.loc[training_indices], label = full_training_data_y.loc[training_indices],
                        categorical_feature = cat_features, free_raw_data = False)
valid_data = lgbm.Dataset(full_training_data_x.loc[validation_indices], label = full_training_data_y.loc[validation_indices],
                        categorical_feature = cat_features, free_raw_data = False)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [10]:
#build the lightgbm model
params = {
          "objective" : "poisson",
          "metric" :"rmse",
          "force_row_wise" : True,
          "learning_rate" : 0.075,
          "sub_row" : 0.75,
          "bagging_freq" : 1,
          "lambda_l2" : 0.1,
          "metric": ["rmse"],
          'verbosity': 1,
          'num_iterations' : 1200,
          'num_leaves': 128,
          "min_data_in_leaf": 100,
         }

# Train LightGBM model
m_lgb = lgbm.train(params, training_data, valid_sets = [valid_data], verbose_eval = 20)

# save the model if needed
m_lgb.save_model("model.lgb")

/home/hhew0002/anaconda3/envs/trmf/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/hhew0002/anaconda3/envs/trmf/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[20]	valid_0's rmse: 2.11949
[40]	valid_0's rmse: 2.14689
[60]	valid_0's rmse: 2.22397
[80]	valid_0's rmse: 2.23508
[100]	valid_0's rmse: 2.23461
[120]	valid_0's rmse: 2.22925
[140]	valid_0's rmse: 2.26143
[160]	valid_0's rmse: 2.27885
[180]	valid_0's rmse: 2.27992
[200]	valid_0's rmse: 2.29394
[220]	valid_0's rmse: 2.28279
[240]	valid_0's rmse: 2.27733
[260]	valid_0's rmse: 2.29683
[280]	valid_0's rmse: 2.28807
[300]	valid_0's rmse: 2.28637
[320]	valid_0's rmse: 2.28742
[340]	valid_0's rmse: 2.30995
[360]	valid_0's rmse: 2.30288
[380]	valid_0's rmse: 2.31598
[400]	valid_0's rmse: 2.32085
[420]	valid_0's rmse: 2.31381
[440]	valid_0's rmse: 2.30247
[460]	valid_0's rmse: 2.29622
[480]	valid_0's rmse: 2.30084
[500]	valid_0's rmse: 2.31781
[520]	valid_0's rmse: 2.31985
[540]	valid_0's rmse: 2.32113
[560]	valid_0's rmse: 2.30873
[580]	valid_0's rmse: 2.30688
[600]	valid_0's rmse: 2.29831
[620]	valid_0's rmse: 2.29615
[640]	valid_0's rmse: 2.30124
[660]	valid_0's rmse: 2.30639
[680]	valid_0'

In [13]:
# create the initial testing data
lag_size = 10
testing_data = sales_data.iloc[0:10, -lag_size:]
testing_data['cat_id'] = sales_data.cat_id.iloc[0:10]
testing_data['state_id'] = sales_data.state_id.iloc[0:10]
testing_data['id'] = sales_data.id.iloc[0:10]

for day in range(1914, 1942):
    testing_data[f"d_{day}"] = np.nan

testing_data = pd.melt(testing_data, id_vars=['id', 'cat_id', 'state_id'], var_name="d", value_name="sales")

# embed the sales data to have a lag length of 10
for lags in range(1,11):
    testing_data['lag' + str(lags)] = testing_data[["id","sales"]].groupby("id")["sales"].shift(lags)

# merge the calendar data into the sales data
testing_data = testing_data.merge(calendar_data, on="d", copy=False)

# merge the price data into the sales data
testing_data = testing_data.merge(price_data_melted, on=["id", "d"], copy=False)

testing_data['snap'] = testing_data.apply (lambda row: add_snap(row), axis=1)
testing_data = testing_data.drop(columns=["snap_CA", "snap_WI", "snap_TX"])

In [21]:
# perform the recursive forecasting

cols = [f"F{i}" for i in range(1,29)]
for prediction_point in range(1914, 1942):
    current_testing_data = testing_data[testing_data["d"].str.split("d_").str[1].astype(int) == prediction_point]
    current_testing_data = current_testing_data.drop(columns=["id", "d", "sales"])
    prediction = 1.028 * m_lgb.predict(current_testing_data)

    # add the most recent prediction back to the testing data
    testing_data.loc[testing_data["d"].str.split("d_").str[1].astype(int) == prediction_point, "sales"] = prediction

    # recreate the lags
    for lags in range(1,11):
        testing_data['lag' + str(lags)] = testing_data[["id","sales"]].groupby("id")["sales"].shift(lags)


sub = testing_data.loc[testing_data["d"].str.split("d_").str[1].astype(int) >= 1914, ["id", "sales"]].copy()
sub["F"] = [f"F{rank}" for rank in sub.groupby("id")["id"].cumcount()+1]
sub = sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
sub.fillna(0., inplace = True)
sub.sort_values("id", inplace = True)
sub.reset_index(drop=True, inplace = True)
sub.to_csv("submission.csv",index=False)
# make the final submission as a weighted average of the three results using three different alphas


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
8   NaN
9   NaN
Name: value, dtype: float64